In [1]:
from pytrends.request import TrendReq
import pandas as pd
import numpy as np
import time, gc
pd.set_option('display.max_rows', 10)

In [2]:
# timeframe = '2020-01-18 2020-04-01'
timeframe = 'today 3-m'

In [3]:
codes = pd.read_csv('google_two_letter.csv')
codes['Province_State'] = codes['Province_State'].fillna('')
codes

Country_Region Province_State country_code
0           Afghanistan                          AF
1               Albania                          AL
2               Algeria                          DZ
3               Andorra                          AD
4                Angola                          AO
..                  ...            ...          ...
308             Vietnam                          VN
309  West Bank and Gaza                          ZA
310      Western Sahara                          EH
311              Zambia                          ZM
312            Zimbabwe                          ZW

[313 rows x 3 columns]

In [4]:
# Login to Google. Only need to run this once, the rest of requests will use the same session.
pytrend = TrendReq()

In [5]:
# Create payload and capture API tokens. Only needed for interest_over_time(), interest_by_region() & related_queries()
kw = ['covid-19', 'covid19', 'coronavirus']
# pytrend.build_payload(kw_list=kw)

In [6]:
# timeframe = '2020-01-18 2020-04-10'
timeframe = 'today 3-m'

In [7]:
timeframe

'today 3-m'

In [8]:
c = 'AF'
# c = 'MK'
pytrend.build_payload(kw, cat=0, timeframe=timeframe, geo=c, gprop='')
# pytrend.build_payload(kw, cat=0, timeframe='today 3-m', geo=c, gprop='')
df = pytrend.interest_over_time()
df.head(), df.tail(), df.shape

(            covid-19  covid19  coronavirus  isPartial
 date                                                 
 2020-01-15         0        0            0      False
 2020-01-16         0        0            0      False
 2020-01-17         0        0            0      False
 2020-01-18         0        0            0      False
 2020-01-19         0        0            0      False,
             covid-19  covid19  coronavirus  isPartial
 date                                                 
 2020-04-08         4        3           77      False
 2020-04-09         4        3           84      False
 2020-04-10         5        0           76      False
 2020-04-11         3        0           64      False
 2020-04-12        10        0           61      False,
 (89, 4))

In [9]:
# loop over countries and collect interest over time dataframes
# can be run more than once in case of failures
if not 'iot' in locals(): iot = {}
fail = []
for index, row in codes.iterrows():
    c = row['country_code']
    if isinstance(c,str)==False: continue
    if c in iot: continue
    try:
        pytrend.build_payload(kw, cat=0, timeframe=timeframe, geo=c, gprop='')
        df = pytrend.interest_over_time()
        df['Country_Region'] = row['Country_Region']
        df['Province_State'] = row['Province_State']
        df['country_code'] = c
        iot[c] = df.reset_index()
        print(index, c, df.shape)
        time.sleep(2)
        gc.collect()
    except:
        print('*** fail:',c)
        fail.append(c)
        time.sleep(2)
        gc.collect()
        continue
        
print('succeeded',len(iot))
print('failed',fail)

0 AF (89, 7)
1 AL (89, 7)
2 DZ (89, 7)
3 AD (89, 7)
4 AO (89, 7)
5 AG (89, 7)
6 AR (89, 7)
7 AM (89, 7)
8 AU-ACT (89, 7)
9 AU-NSW (89, 7)
10 AU-NT (89, 7)
11 AU-QLD (89, 7)
12 AU-SA (89, 7)
13 AU-TAS (89, 7)
14 AU-VIC (89, 7)
15 AU-WA (89, 7)
16 AT (89, 7)
17 AZ (89, 7)
18 BS (89, 7)
19 BH (89, 7)
20 BD (89, 7)
21 BB (89, 7)
22 BY (89, 7)
23 BE (89, 7)
24 BZ (89, 7)
25 BJ (89, 7)
26 BT (89, 7)
27 BO (89, 7)
28 BA (89, 7)
29 BW (89, 7)
30 BR (89, 7)
31 BN (89, 7)
32 BG (89, 7)
33 BF (89, 7)
34 MM (89, 7)
35 BI (89, 7)
36 CV (89, 7)
37 KH (89, 7)
38 CM (89, 7)
39 CA-AB (89, 7)
40 CA-BC (89, 7)
41 CA-MB (89, 7)
42 CA-NB (89, 7)
43 CA-NL (89, 7)
44 CA-NT (89, 7)
45 CA-NS (89, 7)
46 CA-ON (89, 7)
47 CA-PE (89, 7)
48 CA-QC (89, 7)
49 CA-SK (89, 7)
50 CA-YT (89, 7)
51 CF (89, 7)
52 TD (89, 7)
53 CL (89, 7)
54 CN-34 (89, 7)
55 CN-11 (89, 7)
56 CN-50 (89, 7)
57 CN-35 (89, 7)
58 CN-62 (89, 7)
59 CN-44 (89, 7)
60 CN-45 (89, 7)
61 CN-52 (89, 7)
62 CN-46 (89, 7)
63 CN-13 (89, 7)
64 CN-23 (89, 7)
65

In [10]:
# create large dataframe of all scrapes
all = pd.concat(iot)
all.shape

(26700, 9)

In [11]:
all.head()

date  covid-19  covid19  coronavirus isPartial Country_Region  \
AF 0 2020-01-15       0.0      0.0          0.0     False    Afghanistan   
   1 2020-01-16       0.0      0.0          0.0     False    Afghanistan   
   2 2020-01-17       0.0      0.0          0.0     False    Afghanistan   
   3 2020-01-18       0.0      0.0          0.0     False    Afghanistan   
   4 2020-01-19       0.0      0.0          0.0     False    Afghanistan   

     Province_State country_code index  
AF 0                          AF   NaN  
   1                          AF   NaN  
   2                          AF   NaN  
   3                          AF   NaN  
   4                          AF   NaN

In [12]:
all.drop(['index','isPartial'], axis=1, inplace=True)
all.head()

date  covid-19  covid19  coronavirus Country_Region Province_State  \
AF 0 2020-01-15       0.0      0.0          0.0    Afghanistan                  
   1 2020-01-16       0.0      0.0          0.0    Afghanistan                  
   2 2020-01-17       0.0      0.0          0.0    Afghanistan                  
   3 2020-01-18       0.0      0.0          0.0    Afghanistan                  
   4 2020-01-19       0.0      0.0          0.0    Afghanistan                  

     country_code  
AF 0           AF  
   1           AF  
   2           AF  
   3           AF  
   4           AF

In [13]:
all = all.reset_index(drop=True)
all = all.rename(mapper={'date':'Date'},axis=1)
all

Date  covid-19  covid19  coronavirus Country_Region  \
0     2020-01-15       0.0      0.0          0.0    Afghanistan   
1     2020-01-16       0.0      0.0          0.0    Afghanistan   
2     2020-01-17       0.0      0.0          0.0    Afghanistan   
3     2020-01-18       0.0      0.0          0.0    Afghanistan   
4     2020-01-19       0.0      0.0          0.0    Afghanistan   
...          ...       ...      ...          ...            ...   
26695 2020-04-08       1.0      5.0         48.0       Zimbabwe   
26696 2020-04-09       4.0      4.0         37.0       Zimbabwe   
26697 2020-04-10       4.0      6.0         49.0       Zimbabwe   
26698 2020-04-11       5.0      1.0         55.0       Zimbabwe   
26699 2020-04-12       4.0      4.0         39.0       Zimbabwe   

      Province_State country_code  
0                              AF  
1                              AF  
2                              AF  
3                              AF  
4                              AF  
...              ...          ...  
26695                          ZW  
26696                          ZW  
26697                          ZW  
26698                          ZW  
26699                          ZW  

[26700 rows x 7 columns]

In [14]:
a1 = all.copy()
a1

Date  covid-19  covid19  coronavirus Country_Region  \
0     2020-01-15       0.0      0.0          0.0    Afghanistan   
1     2020-01-16       0.0      0.0          0.0    Afghanistan   
2     2020-01-17       0.0      0.0          0.0    Afghanistan   
3     2020-01-18       0.0      0.0          0.0    Afghanistan   
4     2020-01-19       0.0      0.0          0.0    Afghanistan   
...          ...       ...      ...          ...            ...   
26695 2020-04-08       1.0      5.0         48.0       Zimbabwe   
26696 2020-04-09       4.0      4.0         37.0       Zimbabwe   
26697 2020-04-10       4.0      6.0         49.0       Zimbabwe   
26698 2020-04-11       5.0      1.0         55.0       Zimbabwe   
26699 2020-04-12       4.0      4.0         39.0       Zimbabwe   

      Province_State country_code  
0                              AF  
1                              AF  
2                              AF  
3                              AF  
4                              AF  
...              ...          ...  
26695                          ZW  
26696                          ZW  
26697                          ZW  
26698                          ZW  
26699                          ZW  

[26700 rows x 7 columns]

In [15]:
fname = 'google_trends.csv'
a1.to_csv(fname, index=False)
print(fname, a1.shape)

google_trends.csv (26700, 7)
